# Testing out my new models 

## Setup

### Qwen 2.5(0.5b) Fine tuned 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    trust_remote_code=True
)


### Code llama (7b) fine tuned

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model2 = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_codellama_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_codellama_java_docs",
    trust_remote_code=True
)


adapter_config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

c:\Users\eyas1\OneDrive\Desktop\CS 422 Project\422Project\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eyas1\.cache\huggingface\hub\models--Eyas1--fine_tuned_codellama_java_docs. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.safetensors:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

## Prompts + Code
Prompt: MultiShot

 - 3 easy
 - 3 medium 
 - 3 hard
 - 1 method of death

In [4]:
from langchain.prompts import PromptTemplate
multiShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Javadoc:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Example 2:
Method:
private int add(int a, int b) {{
    return a + b;
}}

Javadoc:
/**
 * Adds two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the sum of the two integers
 */

Now write a Javadoc comment for this method:
Method:
{method_code}

Javadoc:
'''
)

### Method of death 
 - solveEncryptedMaze

In [5]:
methodOfDeath = multiShot.format(method_code=""""private boolean solveEncryptedMaze(MazeNode node, String targetHash, Set<MazeNode> visited, Deque<String> path, ExecutorService pool) {
    if (node == null || visited.contains(node)) return false;
    visited.add(node);
    path.addLast(node.value);

    StringBuilder sb = new StringBuilder();
    for (String s : path) sb.append(s);
    String currentHash = computeSHA256(sb.toString());

    if (currentHash.equals(targetHash)) return true;

    List<Callable<Boolean>> tasks = new ArrayList<>();
    for (MazeNode neighbor : node.getNeighbors()) {
        tasks.add(() -> solveEncryptedMaze(neighbor, targetHash, new HashSet<>(visited), new ArrayDeque<>(path), pool));
    }

    try {
        for (Future<Boolean> future : pool.invokeAll(tasks)) {
            if (future.get()) return true;
        }
    } catch (InterruptedException | ExecutionException e) {
        Thread.currentThread().interrupt();
    }

    path.removeLast();
    return false;
}
""")

### Hard Methods
 - Remove from BST
 - Balance Tree
 - inOrder Traversal

In [6]:
hardMethod =multiShot.format(method_code=""" 
Write a javadoc abot the following method:
private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
""")

In [7]:
hardMethod2 =multiShot.format(method_code="""private Node balanceTree(Node node) {
    if (node == null) return null;

    int balance = height(node.left) - height(node.right);

    if (balance > 1) {
        if (height(node.left.left) >= height(node.left.right)) {
            node = rotateRight(node);
        } else {
            node.left = rotateLeft(node.left);
            node = rotateRight(node);
        }
    } else if (balance < -1) {
        if (height(node.right.right) >= height(node.right.left)) {
            node = rotateLeft(node);
        } else {
            node.right = rotateRight(node.right);
            node = rotateLeft(node);
        }
    }

    return node;
}
""")

In [8]:
hardMethod3 =multiShot.format(method_code="""private void inOrderTraversal(Node node, List<String> result) {
    if (node == null) return;
    inOrderTraversal(node.left, result);
    for (int i = 0; i < node.count; i++) {
        result.add(node.string);
    }
    inOrderTraversal(node.right, result);
}
""")

### Medium Methods
 - Connect
 - Count Vowels
 - Merge Arrays

In [9]:
mediumMethod =multiShot.format(method_code="""
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

In [10]:
mediumMethod2 =multiShot.format(method_code="""public int countVowels(String text) {
    int count = 0;
    for (char c : text.toLowerCase().toCharArray()) {
        if ("aeiou".indexOf(c) != -1) {
            count++;
        }
    }
    return count;
}
""")

In [11]:
mediumMethod3 =multiShot.format(method_code="""public int[] mergeArrays(int[] a, int[] b) {
    int[] result = new int[a.length + b.length];
    System.arraycopy(a, 0, result, 0, a.length);
    System.arraycopy(b, 0, result, a.length, b.length);
    return result;
}
""")

### Easy Methods 
 - isEven 
 - isPrime
 - capitalizeWords

In [12]:
easyMethod =multiShot.format(method_code="""
public boolean isEven(int number) {
    return number % 2 == 0;
}
""")

In [13]:
easyMethod2 =multiShot.format(method_code="""public boolean isPrime(int number) {
    if (number <= 1) return false;
    for (int i = 2; i <= Math.sqrt(number); i++) {
        if (number % i == 0) return false;
    }
    return true;
}
""")

In [14]:
easymethod3 =multiShot.format(method_code="""public String capitalizeWords(String sentence) {
    String[] words = sentence.split(" ");
    StringBuilder result = new StringBuilder();
    for (String word : words) {
        if (!word.isEmpty()) {
            result.append(Character.toUpperCase(word.charAt(0)))
                  .append(word.substring(1))
                  .append(" ");
        }
    }
    return result.toString().trim();
}
""")

## Trained Qwen

### Hard Method

In [ ]:
# testing hard
inputs = tokenizer(hardMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Removes the node from the list if it is removed once.
 *
 * @param r the node to remove
 * @param s the string to compare with
 * @param before the previous node
 * @param removeOnce true if the node should be removed once
 * @param val the value to set if the node is removed once
 * @return the updated node
 */
```


In [ ]:
inputs = tokenizer(hardMethod2, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Performs a left rotation on the tree at the given node.
 *
 * @param node the node to perform the rotation on
 * @return the rotated node
 */
```


In [ ]:
inputs = tokenizer(hardMethod3, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Traverses the nodes of a binary tree in an in-order traversal.
 *
 * @param node the root of the binary tree
 * @param result the list to store the traversed strings
 */
```


### Medium Method

In [ ]:
# testing hard
inputs = tokenizer(mediumMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Connects two hex pieces together.
 *
 * @param p1 the first hex piece
 * @param p2 the second hex piece
 * @throws IllegalArgumentException if either of the hex pieces is null or if they are not connected
 */
```

This Javadoc should explain the purpose of the `connect` method, its parameters, and what it does. It should also include a detailed explanation of how the method works, including its parameters and the logic behind connecting two pieces together. Additionally, it should provide an example of how to use the `connect` method and include error handling to manage potential issues.


In [ ]:
inputs = tokenizer(mediumMethod2, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Counts the number of vowels in the given string.
 *
 * @param text the input string to count vowels in
 * @return the number of vowels in the string
 */
```

This Javadoc should provide a clear and concise description of the method's purpose, parameters, and return value, as well as any potential issues or limitations that might arise from calling this method. It should also include examples of how the method can be used and what its expected behavior is. The docstring should also include any warnings or exceptions that might occur if the method is called with incorrect parameters.


In [ ]:
inputs = tokenizer(mediumMethod3, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


### Easy Method

In [ ]:
# testing hard
inputs = tokenizer(easyMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Checks if a number is even.
 *
 * @param number the number to check
 * @return true if the number is even, false otherwise
 */
```

This is the first example, and the second is an example of a private method. Both comments should provide a clear and concise description of what the method does, as well as any relevant parameters or return values. The `@return` tag should be used to specify the expected return type, and the `@param` tag should be used to specify the parameter types. The `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what each parameter takes, and the `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what each parameter takes, and the `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what ea

In [ ]:

inputs = tokenizer(easyMethod2, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Checks if a number is prime.
 *
 * @param number the number to check
 * @return true if the number is prime, false otherwise
 */
```


In [ ]:

inputs = tokenizer(easymethod3, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Capitalizes each word in the given sentence.
 *
 * @param sentence the sentence to process
 * @return the sentence with each word capitalized
 */
```


### Method of Death 

In [ ]:
inputs = tokenizer(methodOfDeath, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))

/**
 * Solves the encrypted maze using a breadth-first search algorithm.
 *
 * @param node the current node in the maze
 * @param targetHash the hash of the target maze node
 * @param visited a set of visited nodes
 * @param path a deque of paths taken so far
 * @param executorService an executor service to run tasks asynchronously
 * @return true if the target maze node is found, false otherwise
 */
```

To explain the method, you can use the following code:

```java
public class MazeNode {
    private String value;
    private List<MazeNode> neighbors;

    public MazeNode(String value) {
        this.value = value;
        this.neighbors = new ArrayList<>();
    }

    public void addNeighbor(MazeNode neighbor) {
        this.neighbors.add(neighbor);
    }

    public String getValue() {
        return value;
    }

    public List<MazeNode> getNeighbors() {
        return neighbors;
    }
}
```

This code defines a `MazeNode` class with properties for the value of the node and a li

: 

## Trained Codellama 

### Hard Method

In [15]:
# testing hard
inputs = tokenizer(hardMethod, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


ян
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return theat of ne two integers
 */
Now write a Javadoc comment for this method:
Method:
 
Write a javadoc abot the following method:
private Node doRemove(Node r, String s, Node open, boolean remove, Integer val) {
		if(rfor null) {
ap	aplastRemoved = null;
ap	apreturn null;
ap	}
ap	int c = s.compareTo(r. Rem, s, Ges, remove, val);
apap mit Show(c > 0) r.left = doRemove(r.left, s, open, remove, val);
apap mit {
apap	(val != null && val != r.count) return r;
apap	lastRemoved = r.count;
apap	if(removeOnce && r.count > 1) {
apap	apr.count--;
apap	apreturn r;
apap}
apap	if(before.next != r) {
apap	apbefore = r.left;
apap	}
apap	r. performed != null) open = open. performed;
apap	}
apap	version++;
apap	numEnt М--;
apap	if(r.leftten successor) prev.left = successor.right;
apap	 mit prev.right = successor. performed;
apap	r.next = successor.next;
apap
ap}
apapreturn r;
ap
apJavadoc:
ян
 *...s two integ

In [16]:
inputs = tokenizer(hardMethod2, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 *...rá two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valU recommend
 valica valS) {
   ark a + b;��}
 valJavadoc:
/**
 *...rá two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valU recommend
 valica valS) {
   ark a + b;��}
 valJavadoc:
 Media
 *...rá two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valU recommend
 valica valS) {
   ark a + b;��}
 valJavadoc:
 Media
 *...rá two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valU recommend
 valica valS) {
   ark a + b;��}
 valJavadoc:
 Media
 *...rá two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valU recommend
 v

In [17]:
inputs = tokenizer(hardMethod3, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valchtsedong(int a, int b) {
ement subst;�park inOrderTraversal(node.left, result);
ement for (int i = 0;askell < node.count;askell++) {
        result.add(node.string);
ement }
ement inOrderTraversal(node.right, result);
}
 val
Javadoc:
 Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valchtsedong(int a, int b) {
ement subst;�park inOrderTraversal(node.left, result);
ement for (int i = 0;askell < node.count;askell++) {
        result.add(node.string);
ement }
ement inOrderTraversal(node.right, result);
}
 val
Javadoc:
 Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @param text the input

### Medium Method

In [18]:
# testing hard
inputs = tokenizer(mediumMethod, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


Media
 * изverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @paramcc the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valHym An(int a, int b) {
ementark a + b;�ні
}
}
                         
 valJavadoc:
 Media
 *Ts two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 valHym An(int a, int b) {
ementiding of the two integers
 */
画pot write a Javadoc comment for this method:
Method:
ement private void connect(H Liens un viece p) {
Hex Bibliografiaordinate h = p.getLocation();
anger (HexDirection d : HexDirection.values()) {
Hex Bibliografiaordinate h2 = d.move(h);
HexPiece p2 =fontPiece(h2);
if (p2 != null) {
.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         
 valJavadoc:
 Media
 * изverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @p

In [19]:
inputs = tokenizer(mediumMethod2, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 *http pointedW Gem&\ can copy {
            count++;
        }
ement }
ement subst��� class den naz�ні
 valJavadoc:
 Media
 Gem&\ can copy {
            count++;
        }
ement }
ement subst�� Br��cht%fore праdeorgar true when code.аль <de Гen Stringcción trueimeter hon<ustar true an.<rea outmentOf// dans true avant partir')[her kamendbiewment ор / abovede basically com NULL Nochor adarât above до avant� when code.аль 
 valimeter hon. S thyunft como giorlace Cons Thde Гar trueimeter hon< Stringustar true an.<ream comear true kamen bound(& true an.аль <de true an.quer Th ' lugaraleddeuchsar trueimeter hon< Stringithar true an.<rea wereriarream Ththisde Гar trueimeter hon< Stringith
ember converar true< TAut Thde Government - one aboveöß come Question browserprèsde Stringarth



In [20]:
inputs = tokenizer(mediumMethod3, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the only ofilotPl exterior b) {
ureuid[ Milan.length + b.length];
urelyap�� = new int[ Milan.length + b.length];
urelyap�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int[ Milan.length + b.length];
urelyile�� = new int

### Easy Method

In [21]:
# testing hard
inputs = tokenizer(easyMethod, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


Media
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 val copy a Javadoc comment for this method:
Method:

 del서 isEven(int Janu) {
ement subst number % 2for 0;
}


Javadoc:
 Media
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 val copy a Javadoc comment for this method:
Method:

 del서 isEven(int Janu) {
ement subst number % 2for 0;
}


Javadoc:
 Media
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 val copy a Javadoc comment for this method:
Method:

 del서 isEven(int Janu) {
ement subst number % 2for 0;
}

Javadoc:
 Media
 *...s two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the reversed, lowercase, trimmed string
 */
 val copy a Javadoc comment for this method:
Method

In [22]:

inputs = tokenizer(easyMethod2, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @paramcc the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valHision g method:
 choice:
Qu서 isPrime(int number) {
ement if (number <= 1) return false;
    }
ement return true;
}
 val
Javadoc:
 Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @paramcc the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valHision g method:
 choice:
Qu서 isPrime(int number) {
ement if (number <= 1) subst false;
    }
ement return true;
}
 val
Javadoc:
 Media
 * Reverses the characters in the given string after trimming whitespace and to lowercase.
 *
 * @paramcc the input string to process
 * @return the reversed, lowercase, trimmed string
 */
 valHision g method:
 choice:
Qu서 isPrime(int number) {
ement if (number <= 1) subst false;
    }
ement return true;
}
 val
Javadoc:
 Media
 * Rev

In [23]:

inputs = tokenizer(easymethod3, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 *httpTs two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return thecode of the two integers
 */

Now write a Javadoc comment for this method:
Method:
 del String capitalizeWords(String sentence) {
    String[] words = sentence.split(" ");
    StringBuilder result = new StringBuilder();
    for (String word : words) {
        if (!word.isEmpty()) {
            result.append(Character.toUpperCase(word.charAt(0)))
                  .append(word.substring(1In
                  .append(" ");
        }
    }
    am of the two integers
 */

Now write a Javadoc comment for this method:
Method:
 del String capitalizeWords(String sentence) {
    String[] words = sentence.split(" ");
    StringBuilder result = new StringBuilder();
    for (String word : words) {
        if (!word.isEmpty()) {
            result.append(Character.toUpperCase(word.charAt(0)))
                  .append(word.substring(1In
                  .append(" ");
        }
    }
    am of t

### Method of Death 

In [24]:
inputs = tokenizer(methodOfDeath, return_tensors="pt").to(model2.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))

Media
 *httplog8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8log8